In [1]:
import pandas as pd
SALE2VOL = 2

In [2]:
land_df = pd.read_excel("耕地性质.xlsx")
land_name2type = dict(zip(land_df["地块名称"], land_df["地块类型"]))  # 地块名称 -> 地块类型
land_name2area = dict(zip(land_df["地块名称"], land_df["地块面积/亩"]))  # 地块名称 -> 地块面积
print("land_name2type \t A1:", land_name2type["A1"], "\tA2:", land_name2type["A2"])
print("land_name2area \t A1:", land_name2area["A1"], "\tA2:", land_name2area["A2"])

land_name2type 	 A1: 平旱地 	A2: 平旱地
land_name2area 	 A1: 80.0 	A2: 55.0


In [3]:
crop_df = pd.read_excel("农作物性质.xlsx")
crop_name2type = dict(zip(crop_df["作物名称"], crop_df["作物类型"])) # 作物名称 -> 作物类型
crop_name2land = dict(zip(crop_df["作物名称"], crop_df["种植耕地"])) # 作物名称 -> 地块类型
print("crop_name2type \t 黄豆:", crop_name2type["黄豆"], "\t茄子:", crop_name2type["茄子"])
print("crop_name2land \t 黄豆:", crop_name2land["黄豆"], "\t茄子:", crop_name2land["茄子"])

crop_name2type 	 黄豆: 粮食（豆类） 	茄子: 蔬菜
crop_name2land 	 黄豆: 平旱地 梯田 山坡地 	茄子: 水浇地-第一季 普通大棚-第一季 智慧大棚-第一季+第二季


In [4]:
# 计算每种作物的 预需求量
plant_2023 = pd.read_excel("2023年种植数据.xlsx")
crop_name2volume = dict()  # 作物名称 -> 预需求量
for i in range(plant_2023.shape[0]):
    crop_name = plant_2023.loc[i, "作物名称"].strip()
    crop_sale = plant_2023.loc[i, "种植面积/亩"]
    if crop_name not in crop_name2volume:
        crop_name2volume[crop_name] = 0
    crop_name2volume[crop_name] += round(crop_sale * SALE2VOL, 8)

del plant_2023, crop_name, crop_sale

In [5]:
# 计算每种作物 在每种地块类型的 每亩利润
price_2023 = pd.read_excel("2023年销售数据.xlsx")
for i in range(len(price_2023)):
    price_range = price_2023.loc[i, "销售单价/(元/斤)"]
    price_low = price_range.split("-")[0]
    price_high = price_range.split("-")[1]
    price_avg = (float(price_low) + float(price_high)) / 2
    price_2023.loc[i, "均值单价"] = round(price_avg, 4)
    price_2023.loc[i, "均值利润"] = price_2023.loc[i, "亩产量/斤"] * price_avg - price_2023.loc[i, "种植成本/(元/亩)"]
    price_2023.loc[i, "均值利润"] = price_2023.loc[i, "均值利润"].round(4)

del price_range, price_low, price_high, price_avg

In [6]:
# 建立数据库 <作物名称, 地块类型, 季次, 每亩利润>
import sqlite3
conn = sqlite3.connect("price.db")
price_db = conn.cursor()
price_db.execute("DROP TABLE IF EXISTS price")
price_db.execute("CREATE TABLE IF NOT EXISTS price (crop_name TEXT, land_type TEXT, season TEXT, profit REAL)")
for i in range(len(price_2023)):
    crop_name = price_2023.loc[i, "作物名称"].strip()
    land_type = price_2023.loc[i, "地块类型"].strip()
    season = price_2023.loc[i, "种植季次"].strip()
    profit = price_2023.loc[i, "均值利润"]
    price_db.execute("INSERT INTO price VALUES (?, ?, ?, ?)", (crop_name, land_type, season, profit))
    if land_type == "普通大棚":
        price_db.execute("INSERT INTO price VALUES (?, ?, ?, ?)", (crop_name, "智慧大棚", season, profit))

conn.commit()
del price_2023, crop_name, land_type, season, profit

['季次', '地块名', '黄豆', '黑豆', '红豆', '绿豆', '爬豆', '小麦', '玉米', '谷子', '高粱', '黍子',
       '荞麦', '南瓜', '红薯', '莜麦', '大麦', '水稻', '豇豆', '刀豆', '芸豆', '土豆', '西红柿', '茄子',
       '菠菜 ', '青椒', '菜花', '包菜', '油麦菜', '小青菜', '黄瓜', '生菜 ', '辣椒', '空心菜', '黄心菜',
       '芹菜', '大白菜', '白萝卜', '红萝卜', '榆黄菇', '香菇', '白灵菇', '羊肚菌']

In [7]:
# 填写结果表
result_df = pd.read_excel("result1.xlsx")
T_crop_name2volume = crop_name2volume.copy()
T_land_name2area = land_name2area.copy()

Season_Change_Time = 0
for i in range(len(result_df)):
    if result_df.loc[i, "季次"].strip() == "第二季":
        Season_Change_Time = i
        break

for i in range(len(result_df)):
    if i == Season_Change_Time: # 换季 预需求量更新
        T_crop_name2volume = crop_name2volume.copy()
        T_land_name2area = land_name2area.copy()
    land_name = result_df.loc[i, "地块名"].strip()
    land_type = land_name2type[land_name]
    season = result_df.loc[i, "季次"].strip()
    result_df.loc[i, "总利润"] = 0
    price_db.execute("SELECT * FROM price WHERE land_type = ? AND season = ? ORDER BY profit DESC", (land_type, season))
    crops = price_db.fetchall()
    for crop in crops:
        crop_name = crop[0]
        if T_crop_name2volume[crop_name] <= 1e-8:
            continue # 如果该作物的预需求量已经满足，则跳过
        avib_area = min(T_crop_name2volume[crop_name], T_land_name2area[land_name])
        avib_area = round(avib_area, 8)
        result_df.loc[i, crop_name] = avib_area
        T_crop_name2volume[crop_name] -= avib_area
        T_land_name2area[land_name] -= avib_area
        result_df.loc[i, "总利润"] += avib_area * crop[3]
        T_crop_name2volume[crop_name] = round(T_crop_name2volume[crop_name], 8)
        T_land_name2area[land_name] = round(T_land_name2area[land_name], 8)
        if T_land_name2area[land_name] <= 1e-8:
            break # 如果该地块的面积已经分配完毕，则跳出
        
del Season_Change_Time, land_name, land_type, season, crop_name, avib_area, crops, crop


/home/ethan/miniconda3/lib/python3.12/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [8]:
result_df.to_excel("result1_ans.xlsx", index=False)